In [ ]:
from __future__ import division
import cv2
import numpy as np

# Create kernels for morphological operations
kernelOpen = np.ones((5, 5), np.uint8)
kernelClose = np.ones((20, 20), np.uint8)

# Set the path to your image here
image_path = 'test2.jpg'

# Load the image
image = cv2.imread(image_path)
if image is None:
    raise ValueError("Image not found or unable to read.")

# Convert BGR to HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define color ranges and create masks
masks = {
    'red1': cv2.inRange(hsv, np.array([0, 50, 50]), np.array([10, 255, 255])),
    'red2': cv2.inRange(hsv, np.array([170, 50, 50]), np.array([180, 255, 255])),
    'green': cv2.inRange(hsv, np.array([50, 50, 50]), np.array([70, 255, 255])),
    'yellow': cv2.inRange(hsv, np.array([20, 50, 50]), np.array([30, 255, 255]))
}

masks['red'] = masks['red1'] + masks['red2']
maskOpen = cv2.morphologyEx(masks['red'], cv2.MORPH_OPEN, kernelOpen)
maskFinal = cv2.morphologyEx(maskOpen, cv2.MORPH_CLOSE, kernelClose)

# Calculate color percentages
total_pixels = hsv.size / 3  # Number of pixels per channel
color_counts = {color: np.sum(mask == 255) for color, mask in masks.items()}
color_percents = {color: count / total_pixels for color, count in color_counts.items()}

# Determine ripeness based on color proportions
ripeness = "Medium Ripeness"
if color_percents['green'] > 0.5:
    ripeness = "Low Ripeness"
elif color_percents['yellow'] > 0.8:
    ripeness = "High Ripeness"

print("Ripeness:", ripeness)
for color, mask in masks.items():
    cv2.imshow(f'{color.capitalize()} Mask', mask)

# Wait for the ESC key
while True:
    if cv2.waitKey(5) == 27:  # ESC key
        break

# Clean up
cv2.destroyAllWindows()

Ripeness: Medium Ripeness
